In [1]:
import importlib
import inspect
import re
from pathlib import Path


## I used this code to collect all of the functions that are in `bids.modeling.hrf` into a `pybids_hrf_functions.py` and then I made a parallel `nilearn_hrf_functions.py`.  To unify the formatting, I just ran the files through my Ruff settings and then it was easier to use the file comparison tool on VSCode to spot any differences.

In [2]:
def get_function_source(module_name, function_name):
    """Import a function from a module and return its cleaned source."""
    mod = importlib.import_module(module_name)
    func = getattr(mod, function_name)

    # Get raw source code
    source = inspect.getsource(func)

    # Remove comments (but keep indentation)
    source = re.sub(r'^\s*#.*', '', source, flags=re.MULTILINE)

    # Remove extra blank lines
    while '\n\n\n' in source:
        source = source.replace('\n\n\n', '\n\n')

    return source

In [3]:
# Get all functions defined in bids.modeling.hrf
bids_mod = importlib.import_module('bids.modeling.hrf')
function_list = [
    name
    for name, obj in inspect.getmembers(bids_mod, inspect.isfunction)
    if inspect.getmodule(obj) == bids_mod
]

print(f'Found {len(function_list)} functions in bids.modeling.hrf')


Found 13 functions in bids.modeling.hrf


In [4]:
src_pybids = get_function_source('bids.modeling.hrf', '_gamma_difference_hrf')
src_nilearn = get_function_source(
    'nilearn.glm.first_level.hemodynamic_models', '_gamma_difference_hrf'
)


In [5]:
# Output files
pybids_out = Path('pybids_hrf_functions_new.py')
nilearn_out = Path('nilearn_hrf_functions_new.py')
pybids_out.write_text('')
nilearn_out.write_text('')

# Loop over functions and extract from both packages
for func_name in function_list:
    try:
        src_pybids = get_function_source('bids.modeling.hrf', func_name)
        pybids_out.write_text(pybids_out.read_text() + '\n\n' + src_pybids)
    except (AttributeError, OSError):
        pass

    try:
        src_nilearn = get_function_source(
            'nilearn.glm.first_level.hemodynamic_models', func_name
        )
        nilearn_out.write_text(nilearn_out.read_text() + '\n\n' + src_nilearn)
    except (AttributeError, OSError):
        # Function not in nilearn
        pass